# Genetic Programming for Symbolic Programming

In this question, your task is to build a GP system to automatically evolve a number of genetic programs for the following regression problem:

You can use a GP library. You should:

- Determine and describe the terminal set and the function set.

- Design the fitness cases and fitness function.

- Set the necessary parameters, such as population size, max tree depth, termination criteria, crossover and mutation rates.

- Run the GP system for 3 times with different random seeds. Report the best genetic programs (their structure and performance) of each of the 3 runs. Do your observations, discussions and draw your conclusions